In [66]:
import socket
import pandas as pd

In [67]:
filename = 'test'
data_points = 126

In [68]:
# Fonction pour convertir une chaîne hexadécimale en une liste de valeurs décimales
def hex_to_dec(hex_str):
    hex_value = '0x' + hex_str.replace(' ', '')
    # Convertir en décimal
    dec_value = int(hex_value, 16)
    return dec_value

def hex_to_dec_little_endian_C2(hex_str):
    # Diviser la chaîne en octets
    hex_values = hex_str.split()
    # Réorganiser les octets en ordre inverse (little-endian)
    hex_values.reverse()
    # Concaténer les octets et ajouter le préfixe '0x'
    # Prendre les 2,5 derniers octets (20 bits de poids faible)
    hex_value = '0x' + hex_values[1][1] + hex_values[2] + hex_values[3]
    print(hex_value)
    # Convertir en décimal
    dec_value = int(hex_value, 16)
    dec_value -= (1 << 20)
    return dec_value

def hex_to_dec_little_endian(hex_str):
    # Diviser la chaîne en octets
    hex_values = hex_str.split()
    # Réorganiser les octets en ordre inverse (little-endian)
    hex_values.reverse()
    # Concaténer les octets et ajouter le préfixe '0x'
    # Prendre les 2,5 derniers octets (20 bits de poids faible)
    hex_value = '0x' + hex_values[0] + hex_values[1] + hex_values[2] + hex_values[3]
    print(hex_value)
    # Convertir en décimal
    dec_value = int(hex_value, 16)
    return dec_value

# Get data

In [69]:
def send_hex_data_and_receive(ip, port, hex_data, timeout=10):
    # Convertir la chaîne hexadécimale en bytes
    data = bytes.fromhex(hex_data)

    # Créer un socket TCP/IP
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            # Se connecter au serveur
            s.connect((ip, port))
            print(f"Connecté à {ip} sur le port {port}")

            # Envoyer les données
            s.sendall(data)
            print(f"Données envoyées: {hex_data}")

            # Définir le timeout pour la réception
            s.settimeout(timeout)

            # Recevoir plusieurs paquets de 1040 bytes
            total_data = b''
            while True:
                try:
                    response = b''
                    response = s.recv(1050)
                    if not response:
                        # Si la réponse est vide, arrêter la réception
                        break
                    total_data += response
                    print(f"Paquet reçu de {len(response)} bytes")
                    print(f"Réponse complète reçue: {response.decode('ascii', errors='ignore')}")
                    with open(filename + '.bin', 'ab') as file:
                        file.write(response)
                except socket.timeout:
                    print("Timeout atteint, fin de la réception des données.")
                    break

            # Afficher la réponse reçue en ASCII
            print(f"Réponse complète reçue: {total_data.decode('ascii', errors='ignore')}")

        except Exception as e:
            print(f"Une erreur est survenue: {e}")
        finally:
            # Fermer la connexion proprement
            try:
                s.close()
                print("Connexion fermée")
            except Exception as e:
                print(f"Une erreur est survenue lors de la fermeture de la connexion: {e}")

In [70]:
# Définir les paramètres de connexion
ip_address = '192.168.1.38'
port_number = 66
Hello = '7F 41 50 01 00 92 7e'
Get_smith = '7f 41 11 01 00 64 65 66 61 75\
6c 74 2e 74 72 61 00 8f 54 18 1d 90 54 18 1d 50\
82 16 1d 00 00 e3 1b 00 00 00 00 fc ec 55 07 46\
f1 a1 77 00 00 00 00 00 00 00 00 00 00 00 00 88\
f3 55 07 90 54 18 1d 00 00 00 00 48 ed 55 07 1b\
3c 0f 75 00 00 e3 1b 00 00 00 00 3a 3c 0f 75 89\
5a 5a 01 88 f3 55 07 e0 55 5c 00 00 00 00 00 88\
f3 55 07 88 f3 55 07 00 00 00 00 46 10 2b 00 10\
ed 55 07 d7 8a c6 76 14 ee 55 07 00 00 00 00 f1\
7e'

# Envoyer les données hexadécimales
#send_hex_data(ip_address, port_number, Hello)
# Envoyer les données hexadécimales
send_hex_data_and_receive(ip_address, port_number, Get_smith)

Connecté à 192.168.1.38 sur le port 66
Données envoyées: 7f 41 11 01 00 64 65 66 61 756c 74 2e 74 72 61 00 8f 54 18 1d 90 54 18 1d 5082 16 1d 00 00 e3 1b 00 00 00 00 fc ec 55 07 46f1 a1 77 00 00 00 00 00 00 00 00 00 00 00 00 88f3 55 07 90 54 18 1d 00 00 00 00 48 ed 55 07 1b3c 0f 75 00 00 e3 1b 00 00 00 00 3a 3c 0f 75 895a 5a 01 88 f3 55 07 e0 55 5c 00 00 00 00 00 88f3 55 07 88 f3 55 07 00 00 00 00 46 10 2b 00 10ed 55 07 d7 8a c6 76 14 ee 55 07 00 00 00 00 f17e
Paquet reçu de 1033 bytes
 20250117191625default.tra                                                                     Smith Chart VSWR)                                                               GenComm             GC747A 112200None  BDE11635  	NV3               21/1/224124703    0      ,  }^   }^   Custom                                  @KL         @Ys    HFC-12D (1/2")                          p    .                     d            X           L      x    @      l    4      `	  	  (


Timeout atteint, fin de la réception des données.
 20250117191625default.tra                                                                     Smith Chart VSWR)                                                               GenComm             GC747A 112200None  BDE11635  	NV3               21/1/224124703    0      ,  }^   }^   Custom                                  @KL         @Ys    HFC-12D (1/2")                          p    .                     d            X           L      x    @      l    4      `	  	  (
  
  
 j {K  >c 
 |
 	 d	  < ,  h  .  q : 
 X E   F  EQ't'n53H\jszJq(?ft"a; #
wAAMuI`2vf}^a8ukn)ff&g[
>Ifx}^]M(Br2r  g  kL % 
 t#  BP      p 4  P7	 f	 "
 
                                                                                                                                                                                                                                                 

In [76]:
def to_little_endian(paquets):
    little_endian_paquets = []
    for paquet in paquets:
        # Diviser le paquet en groupes de 2 caractères (1 octet)
        octets = [paquet[i:i+2] for i in range(0, len(paquet), 2)]
        # Inverser l'ordre des octets pour obtenir le format little-endian
        little_endian_paquet = ''.join(octets[::-1])
        little_endian_paquets.append(little_endian_paquet)
    return little_endian_paquets

def to_decimal(little_endian_paquets):
    decimal_values = []
    for paquet in little_endian_paquets:
        # Convertir le paquet little-endian en entier décimal
        decimal_value = int(paquet, 16)
        decimal_values.append(decimal_value)
    return decimal_values

def to_decimal_20bits(little_endian_paquets):
    decimal_values = []
    for paquet in little_endian_paquets:
        # Convertir le paquet little-endian en entier décimal
        decimal_value = int(paquet[3:], 16)
        decimal_values.append(decimal_value)
    return decimal_values

with open(filename + '.bin', 'rb') as fichier:
    # Se positionner au 551ème octet
    fichier.seek(551)

    frequencies = []
    i = 0
    while(len(frequencies) != data_points):
        frequency = fichier.read(4)
        if len(frequency) < 4:
            break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 4 octets
        if(i != 120):
            frequencies.append(frequency.hex())
        else :
            fichier.read(3)
        i += 1
            
    fichier.seek(4583)
    
    real = []
    while(len(real) != data_points):
        data = fichier.read(4)
        hex_data = data.hex()
        if len(data) < 4:
            break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 4 octets
        if '7d5' in hex_data:
            #print("catched")
            # Retirer "7d" de hex_data
            hex_data = hex_data.replace('7d', '', 1)
            #temporary######
            #real.append("FFFFFFFF")
            ################
            # Lire un octet supplémentaire
            extra_data = fichier.read(1)
            if len(extra_data) < 1:
                break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 1 octet
            # Ajouter l'octet supplémentaire à hex_data
            hex_data += extra_data.hex()
            real.append(hex_data)
        else :
            real.append(data.hex())
    
    fichier.seek(8617)
    
    imag = []
    while(len(imag) != data_points):
        data = fichier.read(4)
        hex_data = data.hex()
        if len(data) < 4:
            break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 4 octets
        if '7d5' in hex_data:
            #print("catched")
            # Retirer "7d" de hex_data
            hex_data = hex_data.replace('7d', '', 1)
            #temporary######
            #imag.append("FFFFFFFF")
            ################
            # Lire un octet supplémentaire
            extra_data = fichier.read(1)
            if len(extra_data) < 1:
                break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 1 octet
            # Ajouter l'octet supplémentaire à hex_data
            hex_data += extra_data.hex()
            imag.append(hex_data)
        else :
            imag.append(hex_data)


little_endian_paquets = to_little_endian(frequencies)
dec_frequencies = to_decimal(little_endian_paquets)
#print(dec_frequencies)

print(real)
little_endian_paquets = to_little_endian(real)
dec_real = to_decimal_20bits(little_endian_paquets)
#print(dec_real)

print(imag)
little_endian_paquets = to_little_endian(imag)
dec_imag = to_decimal_20bits(little_endian_paquets)
#print(dec_imag)

df = pd.DataFrame({
    'frequencies': dec_frequencies,
    'real': dec_real,
    'imag' : dec_imag
})

df.to_csv('output.csv', index=False)

['eafc0e00', 'afce0e00', '78980e00', '0e590e00', '610d0e00', 'c9c10d00', '30710d00', '93160d00', '6ab70c00', '7b4b0c00', '84da0b00', '3e630b00', '90e90a00', 'd37c0a00', 'c8ef0900', 'ff640900', 'c2d30800', 'e93c0800', '2caa0700', 'b8e00600', 'bb680600', 'f4c80500', '2e200500', '84710400', '3ac40300', 'fd0a0300', 'c8580200', '45a00100', 'a6000100', 'ea460000', '4596ffff', '51dffeff', 'f727feff', '8a74fdff', '27c0fcff', '93dbfbff', 'b26efbff', '80a5faff', 'bf35faff', '338ef9ff', 'cde8f8ff', 'b548f8ff', '83c1f7ff', '5ce6f6ff', 'd280f6ff', '90e7f5ff', '066af5ff', 'f0e7f4ff', '9773f4ff', 'f2d8f3ff', 'd598f3ff', '7a4af3ff', '7128f3ff', '023ff3ff', '6604f3ff', 'a3c0f2ff', 'dfa6f2ff', 'c1b9f2ff', 'b774f2ff', '2261f2ff', '3b9df2ff', '1720f3ff', '23caf3ff', 'c6baf4ff', '0afff5ff', '77c4f7ff', '41f1f9ff', 'f382fbff', 'e541fbff', '4db4f9ff', '75cbf7ff', '0749f6ff', '6032f5ff', 'd176f4ff', '66f6f3ff', 'afbbf3ff', '5ed3f3ff', '61b4f3ff', '3888f3ff', '9975f3ff', '6b6ef3ff', 'f099f3ff', 'a8e3f3ff', '07

In [72]:
print(imag[125])

05af0800
